In [2]:
import pandas as pd
import influxdb_client
from influxdb_client import Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pathlib
import subprocess
import os

# Secrets
token = "uTG-Rt2pM9JKTIZGr6ZYgRXcOoaG0wVHXn3ERZsmsdMdQE5wByhR7VSxcVIsATwrfQ8MvRzZ7Se4KdvrBEjHyQ=="
url = "https://iwi-i-influx-db-01.hs-karlsruhe.de:8086"
bucket = "iot_gebaeude_n"
org = "Vorlesung"

# Client erstellen und Read/Write API anlegen
write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_api = write_client.write_api(write_options=SYNCHRONOUS)
query_api = write_client.query_api()



In [3]:
def set_working_dir():
    current_dir = pathlib.Path.cwd()
    git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], cwd=current_dir)
    git_root = pathlib.Path(git_root.decode("utf-8").strip())
    os.chdir(git_root)
    new_dir = pathlib.Path.cwd()
    return new_dir

working_dir = set_working_dir()
data_dir = pathlib.Path(working_dir / "data/hka-aqm-n")
df_dir = pathlib.Path(working_dir / "data/output.parquet")

In [4]:
# Daten lesen
# NOTE: hier wird nur eine Datei von vielen eingelesen!
df_a = pd.read_parquet(df_dir)


In [5]:
df_a

,date_time,device_id,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,rssi,channel_rssi,snr,gateway,channel_index,spreading_factor,bandwidth,f_cnt,room,floor
0,2023-03-18 00:13:10,hka-aqm-n214,19.80,38.14,424,1032,6,0,2.0,6.0,-103,-103,NaN,drag-lps8-01,7,8,125000,6565,214,2
1,2023-03-18 00:28:19,hka-aqm-n214,19.80,38.18,423,999,6,0,2.0,6.0,-102,-102,NaN,drag-lps8-01,6,8,125000,6566,214,2
2,2023-03-18 00:43:25,hka-aqm-n214,19.80,38.18,423,970,6,0,2.0,6.0,-102,-102,NaN,drag-lps8-01,4,8,125000,6567,214,2
3,2023-03-18 00:58:31,hka-aqm-n214,19.78,38.24,417,817,6,2,0.0,0.0,-105,-105,NaN,drag-lps8-01,3,8,125000,6568,214,2
4,2023-03-18 01:13:38,hka-aqm-n214,19.81,38.31,426,883,6,0,2.0,6.0,-101,-101,NaN,drag-lps8-01,1,8,125000,6569,214,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282015,2023-04-15 20:43:02,hka-aqm-n113,20.90,42.05,421,1026,5,0,1.0,0.0,-127,-127,NaN,drag-lps8-01,4,7,125000,3567,113,1
282016,2023-04-15 21:29:16,hka-aqm-n113,20.88,42.57,425,1152,7,2,1.0,0.0,-128,-128,NaN,drag-lps8-01,4,7,125000,3570,113,1
282017,2023-04-15 22:30:29,hka-aqm-n113,20.84,42.18,430,1342,5,0,1.0,0.0,-126,-126,NaN,drag-lps8-01,4,7,125000,3574,113,1
282018,2023-04-15 23:01:16,hka-aqm-n113,20.83,42.57,434,1266,7,2,1.0,0.0,-127,-127,NaN,drag-lps8-01,4,7,125000,3576,113,1


In [6]:
for index, row in df_a.iloc[:5].iterrows():
    timestamp = row['date_time']  # Annahme, dass die Spalte 'timestamp' existiert
    print(f"> ({(index+1):02}/{len(df_a)-1}) Adding Data Point ...")
    for key, value in row.to_dict().items():
        if key != 'date_time':  # Überspringe die Zeitstempel-Spalte
            point = Point(f"{index}_{key}")
            point.tag("sensor", f"{index}")
            point.field("value", value)
            point.time(timestamp)  # Setze den Zeitstempel
            write_api.write(bucket=bucket, org=org, record=point)

> (01/282019) Adding Data Point ...
> (02/282019) Adding Data Point ...
> (03/282019) Adding Data Point ...
> (04/282019) Adding Data Point ...
> (05/282019) Adding Data Point ...


In [8]:
bucket

'iot_gebaeude_n'

In [13]:
# Query erstellen
query = f"""from(bucket: "{bucket}")
 |> range(start: 2023-03-17T23:30:00Z, stop: 2024-05-23T00:00:00Z)"""

# Query ausführen
tables = query_api.query(query, org=org)

# Ergebnis ausgeben
for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 3, 17, 23, 30, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 23, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2023, 3, 18, 0, 13, 10, tzinfo=tzutc()), '_value': 6.0, '_field': 'value', '_measurement': '0_BLE', 'sensor': '0'}
FluxRecord() table: 1, {'result': '_result', 'table': 1, '_start': datetime.datetime(2023, 3, 17, 23, 30, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 23, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2023, 3, 18, 0, 13, 10, tzinfo=tzutc()), '_value': 424, '_field': 'value', '_measurement': '0_CO2', 'sensor': '0'}
FluxRecord() table: 2, {'result': '_result', 'table': 2, '_start': datetime.datetime(2023, 3, 17, 23, 30, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 5, 23, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2023, 3, 18, 0, 13, 10, tzinfo=tzutc()), '_value': 0, '_field': 'value', '_measurement': '0_IR', 'sensor': '0'}
FluxRecord() table: